In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
df = pd.read_csv("transformed_data/merged_data_1.csv", index_col="id")
df.head(10)

,price_total,room_type,is_shared_room,is_private_room,max_guests,is_superhost,is_multi_listing,is_business_listing,cleanliness_score,guest_satisfaction_score,...,longitude,latitude,city,day_type,district,state,country_code,country_name,Crime_Index,Safety_Index
id,,,,,,,,,,,,,,,,,,,,,
0,194.033698,Private room,False,True,2.0,False,1,0,10.0,93.0,...,4.90569,52.41772,Amsterdam,weekday,Gemeente Landsmeer,North Holland,NL,Netherlands,25.7,74.3
1,344.245776,Private room,False,True,4.0,False,0,0,8.0,85.0,...,4.90005,52.37432,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
2,264.101422,Private room,False,True,2.0,False,0,1,9.0,87.0,...,4.97512,52.36103,Amsterdam,weekday,Gemeente Diemen,North Holland,NL,Netherlands,25.7,74.3
3,433.529398,Private room,False,True,4.0,False,0,1,9.0,90.0,...,4.89417,52.37663,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
4,485.552926,Private room,False,True,2.0,True,0,0,10.0,98.0,...,4.90051,52.37508,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
5,552.808567,Private room,False,True,3.0,False,0,0,8.0,100.0,...,4.87699,52.38966,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
6,215.124317,Private room,False,True,2.0,False,0,0,10.0,94.0,...,4.91570,52.38296,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
7,2771.307384,Entire home/apt,False,False,4.0,True,0,0,10.0,100.0,...,4.88467,52.38749,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
8,1001.804420,Entire home/apt,False,False,4.0,False,0,0,9.0,96.0,...,4.86459,52.40175,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3


In [3]:
cities = df['city'].unique().tolist()

In [4]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

data = []

for city in cities:
  driver.get(f"https://www.numbeo.com/cost-of-living/in/{city}?displayCurrency=EUR")

  ''' WE can use this Method or just modify the URL with the Currency Type '''
  # dropdown_element = driver.find_element(By.ID,"displayCurrency")
  # select = Select(dropdown_element)
  # select.select_by_visible_text("EGP")

  wait = WebDriverWait(driver, 15)

  Meal_at_Inexpensive_Restaurant = wait.until(
      EC.presence_of_element_located((
          By.XPATH,"/html/body/div[2]/table/tbody/tr[2]/td[2]/span"
          ))).text
  Monthly_Average_Net_salary = driver.find_element(By.XPATH, "/html/body/div[2]/table/tbody/tr[73]/td[2]/span").text
  Taxi_one_mile = driver.find_element(By.XPATH, "/html/body/div[2]/table/tbody/tr[36]/td[2]/span").text
  Monthly_Basic_Utilities = driver.find_element(By.XPATH, "/html/body/div[2]/table/tbody/tr[43]/td[2]/span").text

  data.append({
    "city": city,
    "Monthly_Average_Net_salary": Monthly_Average_Net_salary,
    "Meal_at_Inexpensive_Restaurant": Meal_at_Inexpensive_Restaurant,
    "Taxi_one_mile": Taxi_one_mile,
    "Monthly_Basic_Utilities": Monthly_Basic_Utilities
    })

driver.quit()

In [5]:
city_data_df = pd.DataFrame(data)
city_data_df

,city,Monthly_Average_Net_salary,Meal_at_Inexpensive_Restaurant,Taxi_one_mile,Monthly_Basic_Utilities
0,Amsterdam,"4,502.10 €",20.50 €,3.00 €,262.00 €
1,Athens,"1,164.49 €",15.00 €,0.95 €,184.56 €
2,Barcelona,"2,102.82 €",15.00 €,1.86 €,159.12 €
3,Berlin,"3,114.06 €",15.00 €,2.80 €,347.11 €
4,Budapest,"1,410.60 €",10.47 €,1.15 €,134.93 €
5,Lisbon,"1,304.65 €",13.75 €,0.90 €,137.93 €
6,London,"4,260.76 €",23.01 €,2.85 €,333.23 €
7,Paris,"3,305.14 €",15.00 €,1.50 €,238.30 €
8,Rome,"1,863.90 €",15.00 €,1.50 €,187.37 €
9,Vienna,"3,135.20 €",18.25 €,1.90 €,291.20 €


In [6]:
import pandas as pd

# example: df is your DataFrame
currency_cols = [
    "Monthly_Average_Net_salary",
    "Meal_at_Inexpensive_Restaurant",
    "Taxi_one_mile",
    "Monthly_Basic_Utilities"
]

def clean_currency(series):
    return (
        series
        .str.replace('\xa0', '', regex=False)   # remove non-breaking space
        .str.replace('€', '', regex=False)      # remove euro sign
        .str.replace(',', '', regex=False)      # remove thousands separator
        .str.strip()
        .astype(float)
    )

city_data_df[currency_cols] = city_data_df[currency_cols].apply(clean_currency)


In [8]:
city_data_df = city_data_df.rename(columns={"Taxi_one_mile": "Taxi_price_per_Km"})

In [9]:
city_data_df

,city,Monthly_Average_Net_salary,Meal_at_Inexpensive_Restaurant,Taxi_price_per_Km,Monthly_Basic_Utilities
0,Amsterdam,4502.10,20.50,3.00,262.00
1,Athens,1164.49,15.00,0.95,184.56
2,Barcelona,2102.82,15.00,1.86,159.12
3,Berlin,3114.06,15.00,2.80,347.11
4,Budapest,1410.60,10.47,1.15,134.93
5,Lisbon,1304.65,13.75,0.90,137.93
6,London,4260.76,23.01,2.85,333.23
7,Paris,3305.14,15.00,1.50,238.30
8,Rome,1863.90,15.00,1.50,187.37
9,Vienna,3135.20,18.25,1.90,291.20


In [11]:
city_data_df.to_csv("External_data/cost_of_living_per_city.csv", index = False)